In [4]:
import pyarrow.parquet as pa
parquet_file = pa.read_table("amazon_fashion_clean_051624.parquet")


In [5]:
import pandas as pd

df = parquet_file.to_pandas()

In [6]:
df.shape

(776976, 18)

In [7]:
df.columns

Index(['title', 'average_rating', 'rating_number', 'features', 'description',
       'price', 'images', 'store', 'details', 'parent_asin',
       'title_review_agg', 'user_id', 'timestamp', 'avg_rating_reviewers',
       'coefvar_rating_reviewers', 'text_agg', 'text_weighted_agg',
       'images_review_cln'],
      dtype='object')

In [21]:
len(df['details'][0])

534

In [19]:
# Normalize the 'details' column into a separate DataFrame
details_df = pd.json_normalize(df['details'])


In [20]:

# Concatenate the 'title' column with the normalized details DataFrame
result_df = pd.concat([df[['title']], details_df], axis=1)

# Display the result
print(result_df)

                                                    title        \
0       CHUVORA 925 Sterling Silver Open Celtic Knot C...  None   
1       XX-Large Slip Stop Single Tread Slipper Socks ...  None   
2       Sterling Silver 3mm Round Cut CZ Tennis Bracel...  None   
3       VERO MONTE 4 Pairs Womens TRULY No Show Socks ...  None   
4       SA106 Womens Rhinestone Jewel Polarized Lens 6...  None   
...                                                   ...   ...   
776971  ForLeatherMore Genuine Leather Lamb Drawing En...  None   
776972  Magnificent Mr. Claus Christmas Suit | Festive...  None   
776973  Workout Longline Padded Yoga Sports Bra Fitnes...  None   
776974  AITFINEISM Classic Slim Fit Bomber Jacket Men ...  None   
776975  DINGANG Lantern Dress for Women Summer Casual ...  None   

       ABPA Partslink Number ASTM Fluid Rating Active Ingredients  \
0                       None              None               None   
1                       None              None           

In [30]:
# Calculate the percentage of missing values in each column
missing_percentage = result_df.isnull().mean() * 100

# Display columns with less than a certain threshold of missing values
threshold = 75 # For example, 50%
significant_columns = missing_percentage[missing_percentage < threshold].index.tolist()
print(f"Columns with less than {threshold}% missing values:", significant_columns)

Columns with less than 75% missing values: ['title', 'Date First Available', 'Is Discontinued By Manufacturer', 'Item model number', 'Package Dimensions']


In [27]:
nonnull_counts = result_df.notnull().sum()

# Aggregate how many columns have x non-null entries
column_count_summary = nonnull_counts.value_counts().sort_index()

# Display the summary
print(column_count_summary)

0           1
1         162
2          53
3          37
4          21
         ... 
285655      1
353521      1
494779      1
749608      1
776976      1
Name: count, Length: 152, dtype: int64


In [35]:
column_count_summary[-1:-100:-1]

776976    1
749608    1
494779    1
353521    1
285655    1
         ..
100       1
99        2
97        1
92        1
87        1
Name: count, Length: 99, dtype: int64

## Task: Extract relevant info from Pinecone

In [2]:
from pinecone import Pinecone
import pyarrow.parquet as pq
import pandas as pd

In [3]:
parquet_file = pq.ParquetFile('data/amazon_fashion_clean_051624.parquet')
print(parquet_file.metadata)


  created_by: parquet-cpp-arrow version 11.0.0
  num_columns: 562
  num_rows: 776976
  num_row_groups: 1
  format_version: 2.6
  serialized_size: 129583


In [4]:
batch_size = 1000
iterator = parquet_file.iter_batches(batch_size=batch_size)

In [5]:
batch = next(iterator)

In [7]:
df = batch.to_pandas()
df.columns

Index(['title', 'average_rating', 'rating_number', 'features', 'description',
       'price', 'images', 'store', 'details', 'parent_asin',
       'title_review_agg', 'user_id', 'timestamp', 'avg_rating_reviewers',
       'coefvar_rating_reviewers', 'text_agg', 'text_weighted_agg',
       'images_review_cln'],
      dtype='object')

In [8]:
# create a dataframe: including, 'parent_asin', 'average_rating', 'rating_number', 'price', 'title', 'description', 'text_weighted_agg'
# include only rows where price is not null and > 0.0; average_rating is not null and > 0.0;

df = df[['parent_asin', 'average_rating', 'rating_number', 'price', 'title', 'description', 'text_weighted_agg']]
df = df.dropna(subset=['price', 'average_rating'])

df = df[df['price'] > 0.0]

df = df[df['average_rating'] > 0.0]

df.head()

,parent_asin,average_rating,rating_number,price,title,description,text_weighted_agg
0,B00LOPVX74,4.3,137,30.00,CHUVORA 925 Sterling Silver Open Celtic Knot C...,This beautiful jewelry would be a great additi...,I think this locket is really pretty. The insi...
1,B07B4JXK8D,4.3,3096,20.99,XX-Large Slip Stop Single Tread Slipper Socks ...,Anyone looking for the best in single tread sl...,Great Very loosely made material. Stretched o...
4,B00XESJTDE,3.9,34,12.95,SA106 Womens Rhinestone Jewel Polarized Lens 6...,Men's anti-glare lens sunglasses 100% UVA & UV...,I LOVE these glasses! They fit perfectly over...
40,B095CHR38Z,3.0,7,10.99,Womens Short Sleeve V Neck Shirts Loose Tops B...,None,"I am 5'7"".. usually wear a 14/16 or 0x/1x.. 38..."
41,B071HMN7K8,4.6,740,11.88,Needzo Ghost and Pumpkin Orange Sequins Witch ...,Fabric / Tulle / Ribbon - Stands approximately...,Purchased for my sister to use on Halloween......


In [9]:
# following the same logic above, create a full dataframe iterated through all the batches

df = pd.DataFrame()

for batch in iterator:
    batch_df = batch.to_pandas()
    batch_df = batch_df[['parent_asin', 'average_rating', 'rating_number', 'price', 'title', 'description', 'text_weighted_agg']]
    batch_df = batch_df.dropna(subset=['price', 'average_rating'])
    batch_df = batch_df[batch_df['price'] > 0.0]
    batch_df = batch_df[batch_df['average_rating'] > 0.0]
    df = pd.concat([df, batch_df])


df.shape

(48613, 7)

In [13]:
df.head()

,parent_asin,average_rating,rating_number,price,title,description,text_weighted_agg
0,B07JB1G8HH,3.0,5,34.99,Restoration Mens Harrison Chelsea Ankle Boot S...,None,Boots arrived in great condition and fit wonde...
1,B07Z8HBBYB,4.1,861,12.92,Clip Holder Buckle Clasp Pin for Poncho Cape C...,Melifluos is a vertically integrated scarf com...,Great value and easy to attach. Looks like it...
4,B08F185Q57,4.1,32,29.99,GloFX Wormhole Kaleidoscope Goggles - Festival...,None,These goggles are great! they are very well m...
9,B07MWYC8YQ,4.4,7,12.95,6Pcs Black Anodized 6mm 8mm 10mm 20g horseshoe...,None,Very hard to screw end on. Screw come loose t...
10,B00A2AQU52,4.3,143,10.49,4 Oz. Kelly Cobbler Leather Dye Dark Brown By ...,None,I received a pair of boots that were purchased...


In [10]:
# keep only the  parent_asin, average_rating, rating_number, price

df_short = df[['parent_asin', 'average_rating', 'rating_number', 'price']].reset_index(drop=True)

In [11]:
df_short.head()

,parent_asin,average_rating,rating_number,price
0,B07JB1G8HH,3.0,5,34.99
1,B07Z8HBBYB,4.1,861,12.92
2,B08F185Q57,4.1,32,29.99
3,B07MWYC8YQ,4.4,7,12.95
4,B00A2AQU52,4.3,143,10.49


### fetch the vectors

In [12]:
pinecone_api_key = "92fd371f-5b1e-415b-bcb4-9a9922514120"
pc = Pinecone(api_key=pinecone_api_key)

text_embed_index_name = "description-embed"
image_embed_index_name = "clip_embed"





In [17]:
def enrich_with_embeddings(df, index_name, batch_size=20, output_column_name='embedding'):
    # Initialize Pinecone with your API key
    pc = Pinecone(api_key=pinecone_api_key)
    pc_index = pc.Index(index_name)

    # Prepare an empty list to store the embeddings
    embedding_list = []

    # Split the dataframe into batches
    for i in range(0, len(df), batch_size):
        batch_df = df[i:i + batch_size]
        embeddings = pc_index.fetch(ids=batch_df['parent_asin'].tolist())

        # Extract ids and their corresponding embeddings
        ids = []
        vectors = []
        for key, value in embeddings['vectors'].items():
            ids.append(key)
            vectors.append(value['values'])

        # Store embeddings in the DataFrame
        batch_df[output_column_name] = batch_df['parent_asin'].apply(lambda x: vectors[ids.index(x)] if x in ids else None)

        # Append the enriched batch to the list
        embedding_list.append(batch_df)

    # Concatenate all enriched batches into a single DataFrame
    enriched_df = pd.concat(embedding_list, ignore_index=True)

    return enriched_df

In [19]:
final_short_df = enrich_with_embeddings(df_short, text_embed_index_name, batch_size=200, output_column_name='description-embed')

C:\Users\Admin\AppData\Local\Temp\ipykernel_17064\3904275763.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch_df[output_column_name] = batch_df['parent_asin'].apply(lambda x: vectors[ids.index(x)] if x in ids else None)
C:\Users\Admin\AppData\Local\Temp\ipykernel_17064\3904275763.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch_df[output_column_name] = batch_df['parent_asin'].apply(lambda x: vectors[ids.index(x)] if x in ids else None)
C:\Users\Admin\AppData\Local\Temp\ipykernel_17064\39

In [ ]:
final_clip_df = enrich_with_embeddings(df_short, image_embed_index_name, batch_size=200, output_column_name='clip-embed')

In [20]:
# save the final dataframe to a csv file

final_short_df.to_csv('data/final_short_df.csv', index=False)

final_short_df.head()

,parent_asin,average_rating,rating_number,price,description-embed
0,B07JB1G8HH,3.0,5,34.99,"[-0.0324714072, 0.0175283104, -0.0140127046, -..."
1,B07Z8HBBYB,4.1,861,12.92,"[-0.0413736627, 0.00502245687, -0.0224917214, ..."
2,B08F185Q57,4.1,32,29.99,"[-0.0326505676, -0.00738119101, -0.0215843357,..."
3,B07MWYC8YQ,4.4,7,12.95,"[-0.00822950341, -0.027486816, -0.0115929851, ..."
4,B00A2AQU52,4.3,143,10.49,"[-0.0163263343, -0.0145302899, -0.0158846825, ..."


In [25]:
# see last 5 rows of the dataframe

final_short_df.tail()

,parent_asin,average_rating,rating_number,price,description-embed
48608,B008M2BWGY,5.0,1,17.95,"[-0.0059869932, -0.0455973148, -0.0159496777, ..."
48609,B08L8JJYFC,5.0,4,8.90,"[-0.0498198904, 0.0341441929, -0.0231682, 0.00..."
48610,B01MRU7VM5,4.5,2,12.99,"[-0.0213032402, 0.0136785191, -0.0150425388, 0..."
48611,B08NXH7KST,4.4,40,5.99,"[-0.037750721, 0.0228010863, -0.0167459063, 0...."
48612,B07NSQC6M6,3.7,56,26.99,"[0.00705363, 0.0112247672, -0.0175662525, 0.02..."


In [23]:
sample_df = df_short[-1:-20]

In [14]:
pc = Pinecone(api_key=pinecone_api_key)
pc_index = pc.Index(text_embed_index_name)
sample_embeddings = pc_index.fetch(ids=sample_df['parent_asin'].tolist())

In [33]:
extracted_dict = {k: v['values'] for k, v in sample_embeddings['vectors'].items()}

In [21]:
aj = pc_index.fetch(ids=["oeffd",'sdf'])

In [22]:
aj

{'namespace': '', 'usage': {'read_units': 1}, 'vectors': {}}

C:\Users\Admin\AppData\Local\Temp\ipykernel_17064\1498135966.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df['embedding'] = embeddings


#### looks like it is costly and more complicated than just do embedding again..